**Instalações necessárias**
- FASE 1: Instalação do python no notebook do colab.

In [ ]:
!pip install -U -q PyDrive

In [ ]:
!pip install -U -q pyarrow

**Importações necessárias**
- FASE 2: Importações das bibliotecas a serem utilizadas.

In [8]:
# importações necessarias
import pandas as pd
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import warnings
import shutil
warnings.filterwarnings('ignore')
import plotly.express as px
import pyarrow as pa
import pyarrow.parquet as pq

ModuleNotFoundError: No module named 'google.colab'

**Autenticação de segurança**
- FASE 3: Verificação de acesso do email INTELI para acessar a base de dados fornecida pelo Banco Pan. Importante para a segurança dos dados.

In [ ]:
# Autenticação para acesso aos dados
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

**Download dos Dados**
- FASE 4: Realização do download dos dados da base para esse notebook do Colab, para a visualização e manipulação.

In [3]:
# Download dos dados para esta máquina virtual
downloaded = drive.CreateFile({'id': '1nYYM_5xH6bWL5gPqBTx_mt0SCzm4EiuU'})
downloaded.GetContentFile("06120018.zip")
shutil.unpack_archive("06120018.zip", "./dataset")

**Filtro das colunas**
- FASE 5: Escolher todas as colunas que de fato seram utilizado

In [4]:
# Filtro de colunas gerado pelo muricola
# Filtro para as colunas que estão descritas no TAPI - by: muricola

filtered_cols = ['amscHprsovDrivF-1a', 'amscHprsovDrivF-1b',
                'amscHprsovDrivF-2b', 'amscPrsovDrivF-1a',
                'amscPrsovDrivF-1b', 'amscPrsovDrivF-2b',
                'basBleedLowPressF-1a', 'basBleedLowPressF-2b',
                'basBleedLowTempF-1a', 'basBleedLowTempF-2b',
                'basBleedOverPressF-1a', 'basBleedOverPressF-2b',
                'basBleedOverTempF-1a', 'basBleedOverTempF-2b',
                'bleedFavTmCmd-1a', 'bleedFavTmCmd-1b',
                'bleedFavTmCmd-2a', 'bleedFavTmCmd-2b', 'bleedFavTmFbk-1a',
                'bleedFavTmFbk-1b', 'bleedFavTmFbk-2b', 'bleedHprsovCmdStatus-1a',
                'bleedHprsovCmdStatus-1b', 'bleedHprsovCmdStatus-2a',
                'bleedHprsovCmdStatus-2b', 'bleedHprsovOpPosStatus-1a',
                'bleedHprsovOpPosStatus-1b', 'bleedHprsovOpPosStatus-2a',
                'bleedHprsovOpPosStatus-2b', 'bleedMonPress-1a',
                'bleedMonPress-1b', 'bleedMonPress-2a', 'bleedMonPress-2b',
                'bleedOnStatus-1a', 'bleedOnStatus-1b', 'bleedOnStatus-2b',
                'bleedOverpressCas-2a', 'bleedOverpressCas-2b',
                'bleedPrecoolDiffPress-1a', 'bleedPrecoolDiffPress-1b',
                'bleedPrecoolDiffPress-2a', 'bleedPrecoolDiffPress-2b',
                'bleedPrsovClPosStatus-1a', 'bleedPrsovClPosStatus-2a',
                'bleedPrsovFbk-1a']

**Converter as colunas**
- FASE 6: Converter todas as colunas do tipo 64 para 32, diminuindo o tamanho do dataset

In [5]:
def convert_columns(df_filtered):
    for coluna in df_filtered.columns:
        if df_filtered[coluna].dtype == 'float64':
            df_filtered[coluna] = df_filtered[coluna].astype('float32')

        if df_filtered[coluna].dtype == 'int64':
            df_filtered[coluna] = df_filtered[coluna].astype('int32')

    return df_filtered

**União dos dataset**
- FASE 7: Passar por todos e uni-los em um só

In [ ]:
df = pd.DataFrame()
df_number = 0
for _, _, arquivos in os.walk('./dataset/06120018'):
    #nota: para alterar o tamanho do dataset, basta alterar o valor de 250. Recomendo utilizar o valor de 250 para não estourar o tamanho da RAM do colab
    while(df_number != 250):
            df_read = pd.read_parquet("./dataset/06120018/" + arquivos[df_number] ,engine='auto')
            df_filtered = df_read[filtered_cols]
            df_converted = convert_columns(df_filtered)
            df = pd.concat([df, df_converted], axis=0)
            df_number = df_number + 1
            #print(df_number)

**Exibir dataset**
- FASE 8: Exibição final do novo dataset gerado

In [ ]:
df

**Salvar o novo dataset**
- FASE 9: Salvando o novo dataset na raiz em um nvo arquivo parquet

In [ ]:
df.to_parquet('./dataframe.parquet', index=False)